In [1]:
from __future__ import print_function
import numpy as np
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from astropy.io import fits
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from scipy import stats
import sys
import glob
import json
import seaborn as sns
import os.path
from collections import OrderedDict
from scipy.stats import gaussian_kde
import pandas as pd
from matplotlib.ticker import FormatStrFormatter
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import RandomUnderSampler
from astropy.table import Table
import pandas as pd
from itertools import combinations
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

/home/luisangel/.conda/envs/luis-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-05 16:00:25.068668: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-05 16:00:25.068898: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-05 16:00:25.070851: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-05 16:00:25.097953: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructi

In [2]:
# Read the ASCII file into an Astropy Table
data_gc = Table.read("catalogo_gcs_spectroscopio_splus_detected.dat", format='ascii')

In [3]:
data_gc

NUMBER,ALPHA,DELTA,u,uerr,g,gerr,r,rerr,i,ierr,z,zerr,F378,F378err,F395,F395err,F410,F410err,F430,F430err,F515,F515err,F660,F660err,F861,F861err,fwhm_i,fwhm_psf_i,ellog_i,ellip_i,class_i,spread_i,flux_radius_i,area_i,a_image,b_image,theta_image,kron_radius
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64
16253,52.65439772166957,-34.85515754646527,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,21.509762,0.4309228,99.0,99.0,99.0,99.0,20.985655,0.4293076,3.964676,1.36,1.2672191,0.21087044,0.6723346,-0.009377983,0.7654491,6,0.88541776,0.6987093,62.493454,3.5
16306,52.63961882667077,-34.85291674010414,99.0,99.0,21.505468,0.14491467,99.0,99.0,21.089676,0.12166906,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,21.286818,0.1148665,99.0,99.0,5.630442,1.36,1.0972694,0.08864683,0.06164774,0.007863585,2.1781638,11,0.9900843,0.90231645,77.92344,0.0
16324,52.65275307842115,-34.852162476132385,99.0,99.0,22.298355,0.28719342,21.956337,0.21723434,99.0,99.0,20.88752,0.37852666,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,5.430654,1.36,2.0048974,0.50122136,0.12759903,0.012198187,4.5430984,9,1.2915813,0.64421314,-70.62824,0.0
16364,52.63377244353862,-34.85072293012919,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,21.0796,0.3807831,99.0,99.0,99.0,99.0,3.4195957,1.36,1.7016889,0.4123485,0.48388717,0.00031423918,1.1321247,3,0.7585834,0.44578266,-19.666363,5.918421
16390,52.62834730730939,-34.849198337746614,99.0,99.0,22.041685,0.22105096,21.017162,0.08331954,20.51756,0.07139743,20.51217,0.12940818,99.0,99.0,99.0,99.0,18.82946,0.8450162,99.0,99.0,21.3907,0.27807727,20.945711,0.07998539,20.574175,0.15917358,3.7929711,1.36,1.2955253,0.22811234,0.88868934,0.0020403622,1.6645076,16,1.3120941,1.0127892,-70.39662,0.0
16397,52.626642977094306,-34.84923820813461,99.0,99.0,21.471018,0.48492202,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,2.2012887,1.36,1.2854649,0.22207135,0.49796444,0.0038801704,1.3151855,6,0.8411336,0.65434194,-44.52031,0.0
16458,52.632685126289566,-34.84701960314504,22.502518,0.5979565,99.0,99.0,22.013561,0.23860535,20.885118,0.42604688,21.18469,0.30696318,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,3.9009938,1.36,1.6480353,0.39321685,0.19945201,-0.0007584256,0.8174204,5,0.8476402,0.5143338,-44.52721,5.379644
16468,52.64617379576764,-34.846590308430436,99.0,99.0,22.934105,0.49898762,22.52174,0.34957176,21.769903,0.21682122,19.319366,1.503469,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,5.1380944,1.36,1.8970934,0.47287786,0.34734163,0.017193848,1.0781424,8,1.2750968,0.6721318,17.344378,0.0
16554,52.623203250535354,-34.84303056889895,21.649937,1.8276151,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,20.292194,1.781233,21.858763,0.47966313,99.0,99.0,99.0,99.0,3.4001122,1.36,1.641702,0.39087605,0.38951153,-0.00626068,2.1934,2,0.7410188,0.45137227,69.73203,6.697212


In [4]:
# Contaminats
data_nogc = Table.read("catalogo_contaminates_splus_detected.dat", format='ascii')

In [5]:
data_nogc

NUMBER,ALPHA,DELTA,u,uerr,g,gerr,r,rerr,i,ierr,z,zerr,F378,F378err,F395,F395err,F410,F410err,F430,F430err,F515,F515err,F660,F660err,F861,F861err,fwhm_i,fwhm_psf_i,ellog_i,ellip_i,class_i,spread_i,flux_radius_i,area_i,a_image,b_image,theta_image,kron_radius
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64
3044,42.8470453260897,-31.37096740340893,11.651432,0.00058115856,11.368085,0.00042013926,11.751593,0.00045861208,11.450518,0.00053494854,99.0,99.0,11.214559,0.00069241226,11.151755,0.00091673894,10.712489,0.00066051556,10.656315,0.00060870434,10.291095,0.00045840707,11.636211,0.00070510135,10.723624,0.0006034137,27.459143,1.35,1.2703147,0.21279353,0.9611419,0.038238447,3.8342078,4338,5.156732,4.059413,-87.82511,3.5
9170,42.955755566099235,-31.085048864463147,13.71948,0.0015239846,11.511275,0.0004175872,11.901758,0.0005072665,11.800021,0.0005736633,11.366286,0.00050563744,13.503729,0.0020148547,13.112395,0.0022906233,12.322975,0.0013967738,12.060535,0.0011685397,11.278885,0.0007235725,11.3758335,0.00044918057,10.704217,0.00048655522,21.800894,1.35,1.1450231,0.12665522,0.96872544,0.04455381,3.3162627,2909,4.1713743,3.643048,-78.25398,3.5
9496,42.955829920033665,-31.033536365326555,11.601796,0.0005681798,11.607529,0.0005134674,11.759823,0.00044733495,11.713865,0.00046415016,99.0,99.0,11.206919,0.0006902779,11.14801,0.00091403816,10.587264,0.0006237736,10.514495,0.0005705591,10.128576,0.0004252827,99.0,99.0,99.0,99.0,37.673557,1.35,1.7743977,0.4364285,0.95908266,0.03368139,5.0185037,5172,8.360392,4.7116785,77.561134,3.5
12377,42.4385965998029,-30.919631033640734,11.618564,0.00057179097,11.512144,0.0004381691,11.774399,0.000449017,11.704341,0.00045745587,11.370043,0.00048808075,11.13302,0.0006658354,10.9551935,0.00083449954,10.638807,0.0006371665,10.576302,0.0005870628,10.356766,0.0004714928,11.416972,0.00046090348,10.717414,0.00048309253,20.161589,1.35,1.2353531,0.19051486,0.9636334,0.03978264,3.507265,3549,5.7006006,4.6145515,12.038451,3.5
13053,42.84712002810965,-30.87701052919544,13.087758,0.0011323754,11.452055,0.00040584552,11.827365,0.00046215075,11.747568,0.00045948787,11.299371,0.00044580168,12.866256,0.0014923584,12.618873,0.0018144058,11.837067,0.0011131917,11.677383,0.0009777384,10.955012,0.0006223606,11.376214,0.00044186632,10.721317,0.00048381812,19.554634,1.35,1.2140043,0.17627966,0.96176684,0.04382633,3.6688995,3820,4.808232,3.9606383,87.60515,3.5
15217,42.997889382512625,-30.788077971661718,14.865441,0.0026395104,11.933815,0.00050708075,12.225038,0.00063878973,11.782852,0.0004758198,11.310628,0.00045017365,14.507848,0.0032594122,14.191498,0.0038487022,13.178082,0.0020886664,12.791295,0.0016479644,11.702214,0.0008798248,11.397788,0.0004461687,10.786533,0.00050151895,38.314373,1.35,4.055133,0.75339895,0.9679872,0.04024402,3.0608144,2500,12.853642,3.1697216,62.146217,3.5
15777,42.60498321902373,-30.778911654026768,12.352217,0.00080334,11.597718,0.00044452466,11.805931,0.0004562576,11.737463,0.0004575144,11.453682,0.000495361,11.950987,0.00097247236,11.898987,0.0012935641,11.441607,0.0009246144,11.357834,0.0008439273,10.988905,0.0006314958,11.856978,0.00061559654,10.924177,0.0005300819,15.727633,1.35,1.0117455,0.011609077,0.98535883,0.038009293,2.726821,2513,3.5821443,3.5405588,-80.94744,3.5
16823,41.95335865149609,-30.707116381638897,11.704669,0.00059559965,11.3163595,0.00038187802,11.720003,0.00043900116,11.62538,0.00043471492,11.271993,0.00044205086,11.379596,0.00074634573,11.315477,0.0009864696,10.774082,0.00067814434,10.706626,0.0006238565,10.25462,0.00045004397,11.298214,0.00042705468,10.669746,0.00047865565,28.64709,1.35,1.2525486,0.20162779,0.95989746,0.039631072,4.260776,4347,5.3419285,4.2648473,86.84391,3.5
18373,42.869801817251606,-30.595474104218198,13.

### Checking coincidencis

In [6]:
# convert in pandas
# Convert Astropy Tables to Pandas DataFrames
df_gc = data_gc.to_pandas()
df_nogc = data_nogc.to_pandas()

In [7]:
# Check for coincidences and remove rows in df_nogc that have matching 'NUMBER' in df_gc
df_nogc_filtered = df_nogc[~df_nogc['NUMBER'].isin(df_gc['NUMBER'])]

In [8]:
df_nogc_filtered

,NUMBER,ALPHA,DELTA,u,uerr,g,gerr,r,rerr,i,...,ellog_i,ellip_i,class_i,spread_i,flux_radius_i,area_i,a_image,b_image,theta_image,kron_radius
1,9170,42.955756,-31.085049,13.719480,0.001524,11.511275,0.000418,11.901758,0.000507,11.800021,...,1.145023,0.126655,0.968725,0.044554,3.316263,2909,4.171374,3.643048,-78.253980,3.5
2,9496,42.955830,-31.033536,11.601796,0.000568,11.607529,0.000513,11.759823,0.000447,11.713865,...,1.774398,0.436428,0.959083,0.033681,5.018504,5172,8.360392,4.711678,77.561134,3.5
3,12377,42.438597,-30.919631,11.618564,0.000572,11.512144,0.000438,11.774399,0.000449,11.704341,...,1.235353,0.190515,0.963633,0.039783,3.507265,3549,5.700601,4.614552,12.038451,3.5
4,13053,42.847120,-30.877011,13.087758,0.001132,11.452055,0.000406,11.827365,0.000462,11.747568,...,1.214004,0.176280,0.961767,0.043826,3.668900,3820,4.808232,3.960638,87.605150,3.5
5,15217,42.997889,-30.788078,14.865441,0.002640,11.933815,0.000507,12.225038,0.000639,11.782852,...,4.055133,0.753399,0.967987,0.040244,3.060814,2500,12.853642,3.169722,62.146217,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26113,30928,57.489591,-40.751356,16.036572,0.007804,14.965096,0.003696,14.743249,0.002863,14.748895,...,1.040525,0.038947,0.966167,0.001143,1.505203,337,2.341257,2.250073,55.424076,3.5
26114,30953,57.161989,-40.767419,17.965970,0.021197,16.303978,0.007054,15.721187,0.004564,15.550023,...,1.031283,0.030334,0.968605,0.000731,1.532388,231,2.223293,2.155852,-63.319294,3.5
26115,31085,56.977211,-40.769335,18.106148,0.022911,16.557777,0.008002,16.063084,0.005382,15.940112,...,1.079151,0.073345,0.997819,0.000656,1.519792,169,2.119495,1.964040,85.501940,3.5
26116,31195,57.498078,-40.759755,16.900860,0.012033,15.565959,0.004926,15.082020,0.003361,14.947059,...,1.054073,0.051299,0.976916,0.001178,1.509249,308,2.308868,2.190425,55.974850,3.5


In [9]:
# Addding labels
# Add a new column named 'label' with value 0 to df1
df_gc['label'] = 0

# Add a new column named 'label' with value 1 to df2
df_nogc_filtered['label'] = 1

In [10]:
# Concatenate the two DataFrames vertically
combined_df = pd.concat([df_gc, df_nogc_filtered], ignore_index=True)
len(combined_df)

26523

In [11]:
# Cleaned error
m_err = (combined_df["rerr"] <= 0.3) & (combined_df["gerr"] <= 0.3) & \
        (combined_df["ierr"] <= 0.3) & \
         (combined_df["F430err"] <= 0.3) & \
        (combined_df["F515err"] <= 0.3) & (combined_df["F660err"] <= 0.3) & \
        (combined_df["F861err"] <= 0.3) & (combined_df["zerr"] <= 0.3)



df_cleanErr = combined_df[m_err]
len(df_cleanErr)

16659

In [12]:
# See how many object saty in each class
mask0 = df_cleanErr["label"] == 0
mask1 = df_cleanErr["label"] == 1
print("GC:", len(df_cleanErr[mask0]))
print("No GC:", len(df_cleanErr[mask1]))

GC: 73
No GC: 16586


### Preparing the data

In [13]:
#Selecting columns
columns = ["r",
"g",
"i",
"z",
"F430",
"F515",
"F660",
"F861"]

In [14]:
df_mag = df_cleanErr[columns]
df_mag

,r,g,i,z,F430,F515,F660,F861
18,20.771109,21.634192,20.403067,20.642290,21.397750,21.053720,20.602430,20.654783
19,20.137680,21.452078,19.779497,19.552048,21.201336,20.714169,20.130114,19.531912
25,20.246616,21.061428,20.222399,20.175852,20.992937,20.684935,20.479723,19.814960
28,18.024874,18.803331,17.634941,17.343376,19.499080,18.669243,17.865160,17.431444
38,19.519290,20.294525,19.167469,19.019533,20.706673,20.065145,19.382828,19.034452
...,...,...,...,...,...,...,...,...
26518,14.743249,14.965096,14.748895,14.796816,15.080612,14.838267,14.801393,14.790771
26519,15.721187,16.303978,15.550023,15.479434,16.811510,16.123583,15.673627,15.517522
26520,16.063084,16.557777,15.940112,15.889799,16.958448,16.393812,16.033278,15.924335
26521,15.082020,15.565959,14.947059,14.911592,15.949590,15.412489,15.070141,14.933365


In [15]:
# Generate all combinations of magnitude columns
color_index_pairs = list(combinations(df_mag, 2))
len(color_index_pairs)

28

In [16]:
def calculate_earnings(df, index_pairs):
    for index_pair in index_pairs:
        color_index_name = f"{index_pair[0]} - {index_pair[1]}"
        df.loc[:, color_index_name] = df[index_pair[0]] - df[index_pair[1]]
    return df

In [17]:
df_colors_mag = calculate_earnings(df_mag, color_index_pairs)

In [18]:
df_colors_mag

,r,g,i,z,F430,F515,F660,F861,r - g,r - i,...,z - F430,z - F515,z - F660,z - F861,F430 - F515,F430 - F660,F430 - F861,F515 - F660,F515 - F861,F660 - F861
18,20.771109,21.634192,20.403067,20.642290,21.397750,21.053720,20.602430,20.654783,-0.863083,0.368042,...,-0.755460,-0.411430,0.039860,-0.012493,0.344030,0.795320,0.742967,0.451290,0.398937,-0.052353
19,20.137680,21.452078,19.779497,19.552048,21.201336,20.714169,20.130114,19.531912,-1.314398,0.358183,...,-1.649288,-1.162121,-0.578066,0.020136,0.487167,1.071222,1.669424,0.584055,1.182257,0.598202
25,20.246616,21.061428,20.222399,20.175852,20.992937,20.684935,20.479723,19.814960,-0.814812,0.024217,...,-0.817085,-0.509083,-0.303871,0.360892,0.308002,0.513214,1.177977,0.205212,0.869975,0.664763
28,18.024874,18.803331,17.634941,17.343376,19.499080,18.669243,17.865160,17.431444,-0.778457,0.389933,...,-2.155704,-1.325867,-0.521784,-0.088068,0.829837,1.633920,2.067636,0.804083,1.237799,0.433716
38,19.519290,20.294525,19.167469,19.019533,20.706673,20.065145,19.382828,19.034452,-0.775235,0.351821,...,-1.687140,-1.045612,-0.363295,-0.014919,0.641528,1.323845,1.672221,0.682317,1.030693,0.348376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26518,14.743249,14.965096,14.748895,14.796816,15.080612,14.838267,14.801393,14.790771,-0.221847,-0.005646,...,-0.283796,-0.041451,-0.004577,0.006044,0.242345,0.279219,0.289841,0.036874,0.047496,0.010622
26519,15.721187,16.303978,15.550023,15.479434,16.811510,16.123583,15.673627,15.517522,-0.582791,0.171164,...,-1.332076,-0.644149,-0.194193,-0.038088,0.687927,1.137883,1.293988,0.449956,0.606061,0.156105
26520,16.063084,16.557777,15.940112,15.889799,16.958448,16.393812,16.033278,15.924335,-0.494693,0.122972,...,-1.068649,-0.504013,-0.143479,-0.034536,0.564636,0.925170,1.034113,0.360534,0.469477,0.108943
26521,15.082020,15.565959,14.947059,14.911592,15.949590,15.412489,15.070141,14.933365,-0.483939,0.134961,...,-1.037998,-0.500897,-0.158549,-0.021773,0.537101,0.879449,1.016225,0.342348,0.479124,0.136776


In [19]:
# Drop magniytudes
df_colors = df_colors_mag.drop(columns=columns)
df_colors

,r - g,r - i,r - z,r - F430,r - F515,r - F660,r - F861,g - i,g - z,g - F430,...,z - F430,z - F515,z - F660,z - F861,F430 - F515,F430 - F660,F430 - F861,F515 - F660,F515 - F861,F660 - F861
18,-0.863083,0.368042,0.128819,-0.626641,-0.282611,0.168679,0.116326,1.231125,0.991902,0.236442,...,-0.755460,-0.411430,0.039860,-0.012493,0.344030,0.795320,0.742967,0.451290,0.398937,-0.052353
19,-1.314398,0.358183,0.585632,-1.063656,-0.576489,0.007566,0.605768,1.672581,1.900030,0.250742,...,-1.649288,-1.162121,-0.578066,0.020136,0.487167,1.071222,1.669424,0.584055,1.182257,0.598202
25,-0.814812,0.024217,0.070764,-0.746321,-0.438319,-0.233107,0.431656,0.839029,0.885576,0.068491,...,-0.817085,-0.509083,-0.303871,0.360892,0.308002,0.513214,1.177977,0.205212,0.869975,0.664763
28,-0.778457,0.389933,0.681498,-1.474206,-0.644369,0.159714,0.593430,1.168390,1.459955,-0.695749,...,-2.155704,-1.325867,-0.521784,-0.088068,0.829837,1.633920,2.067636,0.804083,1.237799,0.433716
38,-0.775235,0.351821,0.499757,-1.187383,-0.545855,0.136462,0.484838,1.127056,1.274992,-0.412148,...,-1.687140,-1.045612,-0.363295,-0.014919,0.641528,1.323845,1.672221,0.682317,1.030693,0.348376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26518,-0.221847,-0.005646,-0.053567,-0.337363,-0.095018,-0.058144,-0.047522,0.216201,0.168280,-0.115516,...,-0.283796,-0.041451,-0.004577,0.006044,0.242345,0.279219,0.289841,0.036874,0.047496,0.010622
26519,-0.582791,0.171164,0.241753,-1.090323,-0.402396,0.047560,0.203665,0.753955,0.824544,-0.507532,...,-1.332076,-0.644149,-0.194193,-0.038088,0.687927,1.137883,1.293988,0.449956,0.606061,0.156105
26520,-0.494693,0.122972,0.173285,-0.895364,-0.330728,0.029806,0.138749,0.617665,0.667978,-0.400671,...,-1.068649,-0.504013,-0.143479,-0.034536,0.564636,0.925170,1.034113,0.360534,0.469477,0.108943
26521,-0.483939,0.134961,0.170428,-0.867570,-0.330469,0.011879,0.148655,0.618900,0.654367,-0.383631,...,-1.037998,-0.500897,-0.158549,-0.021773,0.537101,0.879449,1.016225,0.342348,0.479124,0.136776


In [20]:
# Labels
y = df_cleanErr['label']

In [21]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_colors, y, test_size=0.2, random_state=42, stratify=y)

In [22]:
# Apply SMOTE and Tomek links to the training data
smote_tomek = SMOTETomek(random_state=42)
X_train_resampled, y_train_resampled = smote_tomek.fit_resample(X_train, y_train)

In [23]:
# Standardize the data
scaler = StandardScaler()
X_train_resampled = scaler.fit_transform(X_train_resampled)
X_test = scaler.transform(X_test)

In [24]:
# Set up the parameter grid for GridSearchCV with class weights
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': [{0: 1, 1: 500}, {0: 1, 1: 1000}, {0: 1, 1: 1500}]
}

In [25]:
# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)

In [26]:
# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='f1', n_jobs=-1)


In [27]:
# Fit the GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'class_weight': [{0: 1, 1: 500}, {0: 1, 1: 1000},
                                          {0: 1, 1: 1500}],
                         'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200]},
             scoring='f1')

In [28]:
# Best parameters from GridSearchCV
best_params = grid_search.best_params_
print("Best Parameters: ", best_params)

Best Parameters:  {'class_weight': {0: 1, 1: 500}, 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}


In [29]:
# Train the Random Forest model with the best parameters
rf_model = RandomForestClassifier(**best_params, random_state=42)
rf_model.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(class_weight={0: 1, 1: 500}, min_samples_leaf=4,
                       n_estimators=200, random_state=42)

In [30]:
# Predict on the test data
y_pred = rf_model.predict(X_test)

In [31]:
# Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.15      0.27      0.19        15
           1       1.00      0.99      0.99      3317

    accuracy                           0.99      3332
   macro avg       0.57      0.63      0.59      3332
weighted avg       0.99      0.99      0.99      3332

Confusion Matrix:
 [[   4   11]
 [  23 3294]]
